 7. Build your own CNN and try to achieve the highest possible accuracy on MNIST.

In [1]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import ParameterGrid

In [2]:
mnist = tf.keras.datasets.mnist # load dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data() # split dataset into training and testing sets
# normalize the data
x_train, x_test = x_train / 255.0, x_test / 255.0
# reshape the data for the convolutional neural network
x_train= x_train.reshape((-1,28,28,1)).astype('float32')
x_test= x_test.reshape((-1,28,28,1)).astype('float32')
# one hot encode the labels
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [3]:
print(x_train.shape, x_test.shape)

(60000, 28, 28, 1) (10000, 28, 28, 1)


In [4]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1
)
datagen.fit(x_train)


In [5]:
def create_model(filters=(32, 64, 64), dense_units=64, learning_rate=0.001): # function to create a model
    model = models.Sequential([
        layers.Conv2D(filters[0], (3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(filters[1], (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(filters[2], (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(dense_units, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [6]:
# Grid search for hyperparameters
param_grid = {
    'filters': [(32, 64, 64), (64, 128, 128)],
    'dense_units': [64, 128],
    'learning_rate': [0.001, 0.0005],
    'batch_size': [32,64, 128]
}
grid = ParameterGrid(param_grid)


In [7]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [8]:
def train_and_evaluate(params):
    print(f"Training with params: {params}")
    model = create_model(filters=params['filters'], 
                         dense_units=params['dense_units'], 
                         learning_rate=params['learning_rate'])
    history = model.fit(
        datagen.flow(x_train, y_train, batch_size=params['batch_size']),
        validation_data=(x_test, y_test),
        epochs=5,
        callbacks=[early_stopping],
        verbose=2
    )
    _, test_acc = model.evaluate(x_test, y_test, verbose=0)
    return test_acc, model


In [9]:
# Run grid search
best_acc = 0
best_params = None
best_model = None

for params in grid:
    acc, model = train_and_evaluate(params)
    if acc > best_acc:
        best_acc = acc
        best_params = params
        best_model = model

print(f"Best Accuracy: {best_acc} with params: {best_params}")

# Save the best model
best_model.save('best_mnist_model.h5')

Training with params: {'batch_size': 32, 'dense_units': 64, 'filters': (32, 64, 64), 'learning_rate': 0.001}


Epoch 1/5


c:\Program Files\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Program Files\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1875/1875 - 29s - 15ms/step - accuracy: 0.9153 - loss: 0.2704 - val_accuracy: 0.9833 - val_loss: 0.0510
Epoch 2/5
1875/1875 - 24s - 13ms/step - accuracy: 0.9725 - loss: 0.0893 - val_accuracy: 0.9885 - val_loss: 0.0336
Epoch 3/5
1875/1875 - 20s - 11ms/step - accuracy: 0.9791 - loss: 0.0669 - val_accuracy: 0.9895 - val_loss: 0.0312
Epoch 4/5
1875/1875 - 25s - 13ms/step - accuracy: 0.9819 - loss: 0.0598 - val_accuracy: 0.9923 - val_loss: 0.0231
Epoch 5/5
1875/1875 - 23s - 12ms/step - accuracy: 0.9844 - loss: 0.0500 - val_accuracy: 0.9921 - val_loss: 0.0234
Training with params: {'batch_size': 32, 'dense_units': 64, 'filters': (32, 64, 64), 'learning_rate': 0.0005}
Epoch 1/5
1875/1875 - 26s - 14ms/step - accuracy: 0.8845 - loss: 0.3692 - val_accuracy: 0.9785 - val_loss: 0.0737
Epoch 2/5
1875/1875 - 25s - 13ms/step - accuracy: 0.9615 - loss: 0.1246 - val_accuracy: 0.9874 - val_loss: 0.0398
Epoch 3/5
1875/1875 - 24s - 13ms/step - accuracy: 0.9728 - loss: 0.0872 - val_accuracy: 0.9908 - val_l

Best Accuracy: 0.9939000010490417 with params: {'batch_size': 64, 'dense_units': 64, 'filters': (64, 128, 128), 'learning_rate': 0.001}


# 8. Classifying large images using Inception v3.

a. Download some images of various animals. Load them in Python, for example
using the matplotlib.image.mpimg.imread() function. Resize and/or crop
them to 299 × 299 pixels, and ensure that they have just three channels (RGB),
with no transparency channel.

In [12]:
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [13]:
def preprocess_image(image, label):
    image = tf.image.resize(image, [299, 299])  # Redimensionar para 299x299
    image = tf.image.grayscale_to_rgb(image) if image.shape[-1] == 1 else image  # Garantir 3 canais
    image = tf.cast(image, tf.float32) / 255.0  # Normalizar para [0, 1]
    return image, label


In [ ]:
dataset_name = "oxford_iiit_pet"
dataset, info = tfds.load(dataset_name, split="train[:20]", with_info=True, as_supervised=True)

c:\Program Files\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/2 [00:00<?, ? url/s]


In [ ]:
processed_dataset = dataset.map(preprocess_image)

In [ ]:
# visualize some images
batch_size = 5
batched_dataset = processed_dataset.batch(batch_size)
images, labels = next(iter(batched_dataset))
plt.figure(figsize=(10, 10))
for i in range(batch_size):
    ax = plt.subplot(1, batch_size, i + 1)
    plt.imshow(images[i])
    plt.title(f"Label: {labels[i].numpy()}")
    plt.axis("off")
plt.show()

b. Use the Pretrained Inception v3 Model from TensorFlow/Keras API

In [ ]:
from tensorflow.keras.applications import InceptionV3

In [10]:
model = InceptionV3(weights="imagenet")
model.summary()
print("Pretrained model loaded successfully!")

NameError: name 'InceptionV3' is not defined

c. Create the Inception v3 model by calling the inception_v3() function, as
shown below. This must be done within an argument scope created by the
inception_v3_arg_scope() function. 